# Module 4: Fine-Tuning with LoRA (Low-Rank Adaptation)

**Goal**: Train the model to output a specific JSON format.

**Fix applied**: Explicitly disabled BFloat16 and forced model loading in Float16 to prevent Windows AMP errors.

In [1]:
import torch
import warnings
warnings.filterwarnings("ignore")

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
import gc

gc.collect()
torch.cuda.empty_cache()

## 1. Create Data

In [2]:
data = [
    {"text": "User: I loved this movie! \nAssistant: {\"sentiment\": \"positive\", \"score\": 10}"},
    {"text": "User: It was terrible and boring. \nAssistant: {\"sentiment\": \"negative\", \"score\": 1}"},
    {"text": "User: The food was okay, but service was slow. \nAssistant: {\"sentiment\": \"neutral\", \"score\": 5}"},
    {"text": "User: Best day ever! \nAssistant: {\"sentiment\": \"positive\", \"score\": 10}"},
    {"text": "User: I hate rain. \nAssistant: {\"sentiment\": \"negative\", \"score\": 2}"},
    {"text": "User: It was a disaster. \nAssistant: {\"sentiment\": \"negative\", \"score\": 0}"},
    {"text": "User: Absolutely fantastic service. \nAssistant: {\"sentiment\": \"positive\", \"score\": 9}"}
]

dataset = Dataset.from_list(data)
print("Dataset ready:", dataset[0])

Dataset ready: {'text': 'User: I loved this movie! \nAssistant: {"sentiment": "positive", "score": 10}'}


## 2. Load Model (Force Float16)

In [3]:
model_id = "Qwen/Qwen2.5-1.5B-Instruct"

# 1. Configure Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16 # Compute in FP16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# 2. Load Model with explicit torch_dtype=float16
# This ensures the base model config doesn't default to BFloat16
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=bnb_config, 
    device_map="auto",
    dtype=torch.float16 # <--- CRITICAL FIX: Force FP16
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

## 3. LoRA Config

In [4]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705


## 4. Run Training (Strict FP16)

In [5]:
training_args = SFTConfig(
    output_dir="../qwen_finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=1,
    max_steps=10,
    fp16=True,                 # Enable FP16
    bf16=False,                # <--- CRITICAL FIX: Explicitly disable BF16
    save_strategy="no",
    report_to="none",
    optim="paged_adamw_8bit",
    max_length=512,        # Constrain sequence length
    dataset_text_field="text"   # Memory efficient optimizer
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args  # Explicitly specify text column
)

print("Starting training...")
trainer.train()
print("Training complete!")

Adding EOS to train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Starting training...


NotImplementedError: "_amp_foreach_non_finite_check_and_unscale_cuda" not implemented for 'BFloat16'

## 5. Save

In [ ]:
trainer.model.save_pretrained("final_adapter")
print("Adapter saved to 'final_adapter' folder.")